In [38]:
'''
org_data_file_name = '/ml/sougou/user_tag_query.10W.TRAIN'
org_cut_data_file = '/ml/sougou/orgcut_10W.dat'
modelname = "/ml/sougou/w2v_model_10W.model"
'''
org_data_file_name = '/ml/sougou/user_tag_query.2W.TRAIN'
org_cut_data_file = '/ml/sougou/orgcut.dat'
modelname = "/ml/sougou/w2v_model.model"


import io
import os
import re
import jieba.posseg as pseg

#全局数据
IDlist = []
Agelist = []
Genderlist = []
Edulist = []
Datalist = []

Alllist = []

def mycut( strCnText ):
    words = pseg.cut(strCnText)
    nword = []
    #去除一些不关心的词:[c  连词][p  介词][q  量词][t  时间词][m  量词][eng  英文字母]
    notneeds = [ 'c' , 'p' , 'q' , 't' , 'm' , 'eng' ]
    needs = [ 'n' , 'a' , 'ad' , 'an' , 'd' , 'i' , 'v' ]
    for w in words:
        #fp.write( w.word.encode('utf') + " " + w.flag.encode('utf') + '\n' )
        if ( (w.flag not in notneeds) and (len(w.word)>1) ):
        #if ( (w.flag in needs) and (len(w.word)>1) ):
            #通过停用词库去除一些不需要考虑的词汇
            if w.word not in stopwords:
                nword.append(w.word)
            
    return nword

#开始对数据进行读取
#org_cut_data_file = '/ml/sougou/orgcut.dat'
def create_org_cut_data():
    if ( os.path.exists(org_cut_data_file) == True ):
        cget = raw_input("File exist,cover it?(y / n)")
        if ( cget == 'y' ):
            os.remove( org_cut_data_file )
        else:
            return
        
    #创建文件
    print( "开始创建文件....." )
    fp = open( org_cut_data_file , 'w' )
    for i in range( 0 , len(Datalist) ):
    #for i in range( 0 , 10 ):
        if ( i % 100 == 0 ):
            print( "%d / %d" % ( i , len(Datalist) ) )
            fp.flush()
        fp.write( ','.join(mycut(Datalist[i])).encode('utf') + '\n' )
    fp.close()
        
#加载数据
def load_org_cut_data():
    if ( os.path.exists(org_cut_data_file) == False ):
        create_org_cut_data()
    lines = [line.strip().decode('utf', 'ignore') for line in open(org_cut_data_file).readlines()]
    orgcutdata = []
    for line in lines:
        orgcutdata.append( line.split(',') )
    return orgcutdata

#获得词汇词性
def GetFlag(Text):
    try:
        Flag = SogouDict[Text]
    except:
        Flag = None

    return Flag

'''
N		名词
V		动词
ADJ		形容词
ADV		副词
CLAS	量词
ECHO	拟声词
STRU	结构助词
AUX		助词
COOR	并列连词
CONJ	连词
SUFFIX	前缀
PREFIX	后缀
PREP	介词
PRON	代词
QUES	疑问词
NUM		数词
IDIOM	成语
'''
def IsOkWord( word ):
    Flag = GetFlag( word )
    if ( Flag == None ):
        return True
    #("ECHO" in Flag) or  
    if (  ("STRU" in Flag) or ("AUX" in Flag) or ("COOR" in Flag) \
        or ("CONJ" in Flag) or ("SUFFIX" in Flag) or ("PREFIX" in Flag) or ("CLAS" in Flag) \
        or ("PREP" in Flag) or ("PRON" in Flag) or ("QUES" in Flag) or ("NUM" in Flag)):
        return False
    
    return True

#对数据进行二次处理，一方面是将数据改为空格间隔，另一方面是可以对数据进行进一步的处理
def org_to_final_cut_data(org_cut_data):
    cutdata = []
    for words in org_cut_data:
        #这里可以对words进行进一步的调整    

        wordslen = len( words )
        for i in  range(0,wordslen)[::-1]:
            if ( IsOkWord( words[i].encode("utf-8") ) == False ):
                words.pop(i)

        cutdata.append( ' '.join(words) )
        #cutdata.append( ' '.join(set(words)) )
       
    return cutdata


#-------------------------开始
#按行读取数据
#file = open('/ml/sougou/user_tag_query.2W.TRAIN')
file = open(org_data_file_name)
file_lines = file.readlines()
file_linenum = len(file_lines)
file.close()

#age:    0：未知年龄; 1：0-18岁; 2：19-23岁; 3：24-30岁; 4：31-40岁; 5：41-50岁; 6： 51-999岁
#gender: 0：未知1：男性2：女性
#edu:    0：未知学历; 1：博士; 2：硕士; 3：大学生; 4：高中; 5：初中; 6：小学

#通过正则表达式说去数据
pattern = '([0-9A-Z]*)\s([0-6]{1})\s([0-2]{1})\s([0-6]{1})\s(.*)'

for i in range( 0 , file_linenum ):
    m = re.search( pattern , file_lines[i] )
    if m == None:
        print( '[ERROR]bad lines:' , i )
        break
    else:
        IDlist.append(m.group(1))
        Agelist.append(m.group(2))
        Genderlist.append(m.group(3))
        Edulist.append(m.group(4))
        Datalist.append(m.group(5))
        Alllist.append( m.group(2) + m.group(3) + m.group(4) )

print( "读取原始数据%d" % (len(IDlist)) )

#文本处理
#读取停用词
stopwords = [line.strip().decode('utf-8', 'ignore') for line in open('/ml/sougou/stopwords.txt').readlines()]
print( "读取停用词%d" % (len(stopwords)) )

#读取搜狗词库

SogouDict = {}
file = open('/ml/sougou/SogouLabDic.dic')
SogouLabDic = file.readlines()
SogouLabDicLen = len( SogouLabDic )
pattern = '([^\s]*)\s*[0-9]*\s*(.*)'
for i in range( 0 , SogouLabDicLen ):
    m = re.search( pattern , SogouLabDic[i] )
    if m == None:
        print( '[ERROR]bad lines:' , i )
        break
    else:
        SogouDict[m.group(1).decode("gbk",'ignore').encode("utf-8")] = m.group(2)
SogouLabDicLen = len(SogouDict)
print( "读取搜狗词库%d" % SogouLabDicLen )


#处理数据
org_cut_data = load_org_cut_data()
final_cut_data = org_to_final_cut_data( org_cut_data )
print( "读取切分后数据%d条" % len(final_cut_data) )

#print( final_cut_data[0] )

#这里增加一个对性别位置数据的剔除工作

print( "原有数据个数:%d" % (len(org_cut_data)) )
for i in range( 0 , len(org_cut_data) )[::-1]:
    if (Agelist[i] == '0' or  Genderlist[i] == '0' or Edulist[i] == '0'):
    #if ( Genderlist[i] == '0' ):
        IDlist.pop(i)
        Agelist.pop(i)
        Genderlist.pop(i)
        Edulist.pop(i)
        Alllist.pop(i)
        final_cut_data.pop(i)
        org_cut_data.pop(i)

print ( "剔除未知之后数据个数:%d" % (len(org_cut_data)) )


读取原始数据20000
读取停用词2612
读取搜狗词库157202
读取切分后数据20000条
原有数据个数:20000
剔除未知之后数据个数:17663


In [39]:
#切分数据

import scipy as sp  
import numpy as np  
from sklearn.datasets import load_files  
from sklearn.cross_validation import train_test_split  
from sklearn.feature_extraction.text import  TfidfVectorizer

#doc_terms_train, doc_terms_test, y_train, y_test = train_test_split(final_cut_data, Genderlist, test_size = 0.3)  
#doc_terms_train, doc_terms_test, y_train, y_test = train_test_split(final_cut_data, Agelist, test_size = 0.2)
#doc_terms_train, doc_terms_test, y_train, y_test = train_test_split(final_cut_data, Edulist, test_size = 0.2)
doc_terms_train_spl, doc_terms_test_spl, y_train_spl , y_test_spl = train_test_split(final_cut_data, Alllist, test_size = 0.2)
#doc_terms_train, doc_terms_test, y_train, y_test = train_test_split(org_cut_data, Genderlist, test_size = 0.3)  
#doc_terms_train, doc_terms_test, y_train, y_test = train_test_split(final_cut_data, TargetList, test_size = 0.2)

y_train_a = []
y_train_g = []
y_train_e = []

for i in range( 0 , len(y_train_spl) ):
    y_train_a.append( y_train_spl[i][0] )
    y_train_g.append( y_train_spl[i][1] )
    y_train_e.append( y_train_spl[i][2] )

y_test_a = []
y_test_g = []
y_test_e = []
    
for i in range( 0 , len(y_test_spl) ):
    y_test_a.append( y_test_spl[i][0] )
    y_test_g.append( y_test_spl[i][1] )
    y_test_e.append( y_test_spl[i][2] )

y_train = {'a': y_train_a, 'g': y_train_g, 'e': y_train_e}  
y_test = {'a': y_test_a, 'g': y_test_g, 'e': y_test_e}

doc_terms_train = doc_terms_train_spl
doc_terms_test = doc_terms_test_spl

'''
doc_terms_train_a = []
doc_terms_train_g = []
doc_terms_train_e = []
for i in range( 0 , len(doc_terms_train_spl) ):
    doc_terms_train_a.append( doc_terms_train_spl[i] )
    doc_terms_train_g.append( doc_terms_train_spl[i] )
    doc_terms_train_e.append( doc_terms_train_spl[i] )
    
doc_terms_train = {'a': doc_terms_train_a, 'g': doc_terms_train_g, 'e': doc_terms_train_e}  
doc_terms_test = doc_terms_test_spl

for flag in ['a','g','e']:
    print( "train_" + flag + "原有数据个数:%d" % (len(y_train[flag])) )
    for i in range( 0 , len(y_train[flag]) )[::-1]:
        if ( y_train[flag][i] == '0' ):
            y_train[flag].pop(i)
            doc_terms_train[flag].pop(i)
    print( "train_" + flag + "现有数据个数:%d" % (len(y_train[flag])) )

print( "test原有数据个数:%d" % (len(y_test[flag])) )
for i in range( 0 , len(doc_terms_test) )[::-1]:
    if ( y_test['a'][i] == '0' or  y_test['g'][i] == '0' or y_test['e'][i] == '0'):
        y_test['a'].pop(i)
        y_test['g'].pop(i)
        y_test['e'].pop(i)
        doc_terms_test.pop(i)
print( "test现有数据个数:%d" % (len(doc_terms_test)) )
'''
print( "train:%d test:%d" % (len(doc_terms_train) , len(doc_terms_test)) )

train:14130 test:3533


In [40]:

import time
from gensim.models import word2vec

w2vsize = 300

def w2v_transform( model , data ):
    res=np.zeros((len(data),w2vsize))
    for i,line in enumerate(data):
        terms=line.split()
        count=0
        for j,term in enumerate(terms):
            try:#---try失败说明X中有单词不在model中，训练的时候model的模型是min_count的 忽略了一部分单词
                count += 1
                res[i]+=np.array(model[term])
            except:
                1 == 1
        if count!=0:
            res[i]=res[i]/float(count) # 求均值
    return res

def w2v_loadmodel():
    model = word2vec.Word2Vec.load(modelname)
    return model

def w2v_createmodel( data ):
    print '转换数据...'
    datalist = []
    for i in range(0,len(data)):
        datalist.append( data[i].replace( ',' , ' ' ).split() )
    
    print '开始训练model...'
    start = time.clock()
    model = word2vec.Word2Vec(datalist , size=w2vsize, window=100,workers=48)  # 训练模型; 注意参数window 对结果有影响 一般5-100
    end = time.clock()
    
    print "训练完毕: %f s" % (end - start)
    model.save( modelname )
    print '保存完毕'
    
#w2v_createmodel( final_cut_data )
model = w2v_loadmodel()
print model



Word2Vec(vocab=46336, size=300, alpha=0.025)


In [45]:
#转换
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

def mytransform( doc_train , doc_test ):
#    vectorizer_g = TfidfVectorizer(max_df = 0.75 , max_features = 90000 , ngram_range = (1, 2) , norm = 'l2')
#    vectorizer_a = TfidfVectorizer(max_df = 0.75 , max_features = 130000 , ngram_range = (1, 2) , norm = 'l2')
#    vectorizer_e = TfidfVectorizer(max_df = 0.75 , max_features = 170000 , ngram_range = (1, 2) , norm = 'l2')

    vectorizer_g = TfidfVectorizer(max_df = 0.75 , max_features = 100000 , ngram_range = (1, 2) , norm = 'l2')
    vectorizer_a = TfidfVectorizer(max_df = 0.75 , max_features = 130000 , ngram_range = (1, 2) , norm = 'l2')
    vectorizer_e = TfidfVectorizer(max_df = 0.75 , max_features = 170000 , ngram_range = (1, 2) , norm = 'l2')

    print '性别数据矩阵化:'
    x_train_g = vectorizer_g.fit_transform(doc_terms_train)
    x_test_g = vectorizer_g.transform(doc_terms_test)
    print( "max_features = %d" % ( len(vectorizer_g.get_feature_names()) ) )
    
    print '年龄数据矩阵化:'
    x_train_a = vectorizer_a.fit_transform(doc_terms_train)
    x_test_a = vectorizer_a.transform(doc_terms_test)
    print( "max_features = %d" % ( len(vectorizer_a.get_feature_names()) ) )
    
    print '学历数据矩阵化:'
    x_train_e = vectorizer_e.fit_transform(doc_terms_train)
    x_test_e = vectorizer_e.transform(doc_terms_test)
    print( "max_features = %d" % ( len(vectorizer_e.get_feature_names()) ) )
    
    print 'word2vec 模式转化:'
    x_trainW = w2v_transform( model , doc_terms_train )
    print 'word2vec test 模式转化:'
    x_testW = w2v_transform( model , doc_terms_test )
    print( "size = 300" )
    
    return x_train_g , x_test_g , x_train_a , x_test_a , x_train_e , x_test_e , x_trainW , x_testW

x_train = {}
x_test = {}
x_train['g'], x_test['g'] ,x_train['a'] ,x_test['a'] ,x_train['e'] ,x_test['e'] ,x_train['w'] ,x_test['w'] = \
mytransform( doc_terms_train , doc_terms_test )



性别数据矩阵化:
max_features = 100000
年龄数据矩阵化:
max_features = 130000
学历数据矩阵化:
max_features = 170000
word2vec 模式转化:
word2vec test 模式转化:
size = 300


In [46]:
import numpy as np
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

random_rate = 8240

def clf_LogisticRegression( x_train , y_train ):
    from sklearn.linear_model.logistic import LogisticRegression
    model = LogisticRegression(C=2,penalty='l2')
    clf = model.fit(x_train , y_train )
    return clf

def clf_MultinomialNB( x_train , y_train ):
    from sklearn.naive_bayes import MultinomialNB  
    model = MultinomialNB(alpha=0.01)
    clf = model.fit(x_train , y_train )
    return clf

def clf_BernoulliNB( x_train , y_train ):
    from sklearn.naive_bayes import BernoulliNB  
    model = BernoulliNB(alpha=0.01)
    clf = model.fit(x_train , y_train )
    return clf

def clf_SGDClassifier( x_train , y_train ):
    from sklearn.linear_model import SGDClassifier  
    model = SGDClassifier(alpha=5e-05)
    clf = model.fit(x_train , y_train )
    return clf

def clf_KNeighborsClassifier( x_train , y_train ):
    from sklearn.neighbors import KNeighborsClassifier
    model = KNeighborsClassifier()
    clf = model.fit(x_train , y_train )
    return clf

def clf_SVC( x_train , y_train ):
    from sklearn.svm import SVC
    model = SVC( kernel = 'linear' )
    clf = model.fit(x_train , y_train )
    return clf

def clf_DecisionTreeClassifier( x_train , y_train ):
    from sklearn.tree import DecisionTreeClassifier
    model = DecisionTreeClassifier()
    clf = model.fit(x_train , y_train )
    return clf

def clf_LinearSVC( x_train , y_train ):
    from sklearn.svm import LinearSVC
    model = LinearSVC(C=0.1, random_state=random_rate)
    #model = LinearSVC()
    clf = model.fit(x_train , y_train )
    return clf

def clf_RidgeClassifier( x_train , y_train ):
    from sklearn.linear_model import RidgeClassifier
    model = RidgeClassifier(alpha=0.5)
    clf = model.fit(x_train , y_train )
    return clf

def clf_PassiveAggressiveClassifier( x_train , y_train ):
    from sklearn.linear_model import PassiveAggressiveClassifier
    model = PassiveAggressiveClassifier( C = 0.9 , loss = 'hinge' , n_iter = 20 )
    clf = model.fit(x_train , y_train )
    return clf

'''=========================================================================='''
#0.802315681934
def fun_LogisticRegression( x_train , y_train , x_test , y_test ):
    clf = clf_LogisticRegression(x_train , y_train )
    predictions = clf.predict( x_test )
    return precision_score(y_test, predictions, average='micro')
    
#0.801464328282
def fun_MultinomialNB( x_train , y_train , x_test , y_test ):
    clf = clf_MultinomialNB(x_train , y_train )
    predictions = clf.predict( x_test )
    return precision_score(y_test, predictions, average='micro')

#0.811510301379
def fun_BernoulliNB( x_train , y_train , x_test , y_test ):
    clf = clf_BernoulliNB(x_train , y_train )
    predictions = clf.predict( x_test )
    return precision_score(y_test, predictions, average='micro')
    
def fun_SGDClassifier( x_train , y_train , x_test , y_test ):
    clf = clf_SGDClassifier(x_train , y_train )
    predictions = clf.predict( x_test )
    return( precision_score(y_test, predictions, average='micro')   )
    
def fun_KNeighborsClassifier( x_train , y_train , x_test , y_test ):
    clf = clf_KNeighborsClassifier(x_train , y_train )
    predictions = clf.predict( x_test )
    return( precision_score(y_test, predictions, average='micro')   )

def fun_SVC( x_train , y_train , x_test , y_test ):
    clf = clf_SVC(x_train , y_train )
    predictions = clf.predict( x_test )
    return( precision_score(y_test, predictions, average='micro')   )

def fun_DecisionTreeClassifier( x_train , y_train , x_test , y_test ):
    clf = clf_DecisionTreeClassifier(x_train , y_train )
    predictions = clf.predict( x_test )
    return( precision_score(y_test, predictions, average='micro')   )

def fun_LinearSVC( x_train , y_train , x_test , y_test ):
    clf = clf_LinearSVC(x_train , y_train )
    predictions = clf.predict( x_test )
    return( precision_score(y_test, predictions, average='micro')   )
    
def fun_RidgeClassifier( x_train , y_train , x_test , y_test ):
    clf = clf_RidgeClassifier(x_train , y_train )
    predictions = clf.predict( x_test )
    return( precision_score(y_test, predictions, average='micro')   )

def fun_PassiveAggressiveClassifier( x_train , y_train , x_test , y_test ):
    clf = clf_PassiveAggressiveClassifier(x_train , y_train )
    predictions = clf.predict( x_test )
    return( precision_score(y_test, predictions, average='micro')   )


In [47]:
from random import choice
def sub_vote( mod , *lista ):
    dic = {}
    for i in range( 0 , len(lista) ):
        try:
            dic[lista[i]] = dic[lista[i]] + 1
        except:
            dic[lista[i]] = 1

    rec = 0
    pop = []
    for i in range( 0 , len(lista) ):
        try:
            item = max(dic.items(), key=lambda x: x[1])
            if ( rec == 0 ):
                rec = item[1]
                pop.append( item[0] )
                dic.pop( item[0] )
            elif ( rec == item[1] ):
                pop.append( item[0] )
                dic.pop( item[0] )
            else:
                break
        except:
            break
    if mod == 'g':
        if len(pop) != 1:
            return '2'
    elif mod == 'a':
        if len(pop) != 1:
            return max( pop )
    elif mod == 'e':
        if len(pop) != 1:
            return min( pop )

    return choice(pop)

def sub_vote_a( *list ):
    sum = 0
    for i in range( 0 , len(list) ):
        sum += int(list[i])
    return str( int (round( sum / float(len(list)) ) ) )


def fun_vote( xtrain , ytrain , xtest , ytest , mod ):
    print "mod = " + mod
    pre_LR = clf_LogisticRegression( xtrain[mod] , ytrain[mod] ).predict( xtest[mod] )
    #pre_MB = clf_MultinomialNB( xtrain[mod] , ytrain[mod] ).predict( xtest[mod] )
    pre_BB = clf_BernoulliNB( xtrain[mod] , ytrain[mod] ).predict( xtest[mod] )
    pre_SG = clf_SGDClassifier( xtrain[mod] , ytrain[mod] ).predict( xtest[mod] )
    pre_LS = clf_LinearSVC( xtrain[mod] , ytrain[mod] ).predict( xtest[mod] )
    
    
    pre_LRW = clf_LogisticRegression( xtrain['w'] , ytrain[mod] ).predict( xtest['w'] )
    pre_RCW = clf_RidgeClassifier( xtrain['w'] , ytrain[mod] ).predict( xtest['w'] )
    #pre_BBW = clf_BernoulliNB( x_trainW , ytrain ).predict( x_testW )
    #pre_SGW = clf_SGDClassifier( x_trainW , ytrain ).predict( x_testW )
    '''
    0.837279 0.610938 0.632635 0.693617+
    0.848579 0.611899 0.636307 0.698928+
    MultinomialNB:
    0.834002 0.573366 0.595909 0.667759
    BernoulliNB:
    0.832533 0.571162 0.582462 0.662052+
    0.822815 0.539579 0.519295 0.627229
    SGDClassifier:
    0.844454 0.596926 0.622521 0.687967+
    0.803435 0.571106 0.578564 0.651035
    LinearSVC:
    0.841968 0.609469 0.632861 0.694766++
    RidgeClassifier:
    0.805469 0.574326 0.598960 0.659585
    0.846545 0.600542 0.632465 0.693184++
    '''
    predictions = []

    #随机投票
    for i in range( 0 , len(pre_LR) ):
        predictions.append( sub_vote( mod , pre_LR[i],  pre_BB[i] , pre_SG[i] ,pre_LS[i] ,\
                                     pre_LRW[i] ,  pre_RCW[i] ,pre_LS[i] ,  pre_RCW[i] ) )
        #predictions.append( sub_vote( mod , pre_LR[i],  pre_SG[i] , pre_LRW[i] , pre_LS[i] , pre_RCW[i]) )

    return precision_score(ytest[mod], predictions, average='micro')

print( "vote:" )
v_g = fun_vote( x_train , y_train , x_test , y_test , 'g' )
v_a = fun_vote( x_train , y_train , x_test , y_test , 'a' )
v_e = fun_vote( x_train , y_train , x_test , y_test , 'e' )
print( "%f %f %f %f" % ( v_g , v_a , v_e , (v_g + v_a + v_e) / 3.0 ) )

print 'over'

#0.834701 0.577979 0.613360 0.675347
#0.847438 0.589018 0.612794 0.683083
#0.847155 0.587603 0.611378 0.682045

#0.837815 0.584489 0.610812 0.677705
#0.837532 0.585055 0.610246 0.677611

#0.832720 0.584772 0.603170 0.673554
#0.832437 0.581659 0.602887 0.672328

#0.845740 0.591848 0.603736 0.680442


#0.852139 0.616928 0.640375 0.703147

#0.852252 0.610769 0.633934 0.698985

#0.850896 0.614724 0.635968 0.700529

#0.850444 0.614950 0.635855 0.700416

#无贝叶斯
#0.844511 0.610486 0.635573 0.696857

#+BernoulliNB
#0.849370 0.616080 0.637550 0.701000

#+MultinomialNB
#0.849596 0.615119 0.637211 0.700642

vote:
mod = g
mod = a
mod = e
0.814605 0.580810 0.607982 0.667799
over


In [44]:
print( "LogisticRegression:" )
v_g = fun_LogisticRegression( x_train['g'] , y_train['g'] , x_test['g'] , y_test['g'] )
v_a = fun_LogisticRegression( x_train['a'] , y_train['a'] , x_test['a'] , y_test['a'] )
v_e = fun_LogisticRegression( x_train['e'] , y_train['e'] , x_test['e'] , y_test['e'] )
print( "%f %f %f %f" % ( v_g , v_a , v_e , (v_g + v_a + v_e) / 3.0 ) )
v_g = fun_LogisticRegression( x_train['w'] , y_train['g'] , x_test['w'] , y_test['g'] )
v_a = fun_LogisticRegression( x_train['w'] , y_train['a'] , x_test['w'] , y_test['a'] )
v_e = fun_LogisticRegression( x_train['w'] , y_train['e'] , x_test['w'] , y_test['e'] )
print( "%f %f %f %f" % ( v_g , v_a , v_e , (v_g + v_a + v_e) / 3.0 ) )

print( "MultinomialNB:" )
v_g = fun_MultinomialNB( x_train['g'] , y_train['g'] , x_test['g'] , y_test['g'] )
v_a = fun_MultinomialNB( x_train['a'] , y_train['a'] , x_test['a'] , y_test['a'] )
v_e = fun_MultinomialNB( x_train['e'] , y_train['e'] , x_test['e'] , y_test['e'] )
print( "%f %f %f %f" % ( v_g , v_a , v_e , (v_g + v_a + v_e) / 3.0 ) )

print( "BernoulliNB:" )
v_g = fun_BernoulliNB( x_train['g'] , y_train['g'] , x_test['g'] , y_test['g'] )
v_a = fun_BernoulliNB( x_train['a'] , y_train['a'] , x_test['a'] , y_test['a'] )
v_e = fun_BernoulliNB( x_train['e'] , y_train['e'] , x_test['e'] , y_test['e'] )
print( "%f %f %f %f" % ( v_g , v_a , v_e , (v_g + v_a + v_e) / 3.0 ) )
v_g = fun_BernoulliNB( x_train['w'] , y_train['g'] , x_test['w'] , y_test['g'] )
v_a = fun_BernoulliNB( x_train['w'] , y_train['a'] , x_test['w'] , y_test['a'] )
v_e = fun_BernoulliNB( x_train['w'] , y_train['e'] , x_test['w'] , y_test['e'] )
print( "%f %f %f %f" % ( v_g , v_a , v_e , (v_g + v_a + v_e) / 3.0 ) )

print( "SGDClassifier:" )
v_g = fun_SGDClassifier( x_train['g'] , y_train['g'] , x_test['g'] , y_test['g'] )
v_a = fun_SGDClassifier( x_train['a'] , y_train['a'] , x_test['a'] , y_test['a'] )
v_e = fun_SGDClassifier( x_train['e'] , y_train['e'] , x_test['e'] , y_test['e'] )
print( "%f %f %f %f" % ( v_g , v_a , v_e , (v_g + v_a + v_e) / 3.0 ) )
v_g = fun_SGDClassifier( x_train['w'] , y_train['g'] , x_test['w'] , y_test['g'] )
v_a = fun_SGDClassifier( x_train['w'] , y_train['a'] , x_test['w'] , y_test['a'] )
v_e = fun_SGDClassifier( x_train['w'] , y_train['e'] , x_test['w'] , y_test['e'] )
print( "%f %f %f %f" % ( v_g , v_a , v_e , (v_g + v_a + v_e) / 3.0 ) )

print( "LinearSVC:" )
v_g = fun_LinearSVC( x_train['g'] , y_train['g'] , x_test['g'] , y_test['g'] )
v_a = fun_LinearSVC( x_train['a'] , y_train['a'] , x_test['a'] , y_test['a'] )
v_e = fun_LinearSVC( x_train['e'] , y_train['e'] , x_test['e'] , y_test['e'] )
print( "%f %f %f %f" % ( v_g , v_a , v_e , (v_g + v_a + v_e) / 3.0 ) )


print( "RidgeClassifier:" )
v_g = fun_RidgeClassifier( x_train['g'] , y_train_g , x_test['g'] , y_test['g'] )
v_a = fun_RidgeClassifier( x_train['a'] , y_train_a , x_test['a'] , y_test['a'] )
v_e = fun_RidgeClassifier( x_train['e'] , y_train_e , x_test['e'] , y_test['e'] )
print( "%f %f %f %f" % ( v_g , v_a , v_e , (v_g + v_a + v_e) / 3.0 ) )
v_g = fun_RidgeClassifier( x_train['w'] , y_train_g , x_test['w'] , y_test['g'] )
v_a = fun_RidgeClassifier( x_train['w'] , y_train_a , x_test['w'] , y_test['a'] )
v_e = fun_RidgeClassifier( x_train['w'] , y_train_e , x_test['w'] , y_test['e'] )
print( "%f %f %f %f" % ( v_g , v_a , v_e , (v_g + v_a + v_e) / 3.0 ) )

print 'over'

LogisticRegression:
0.800170 0.566657 0.597792 0.654873
0.809793 0.570337 0.605151 0.661761
MultinomialNB:
0.808944 0.569205 0.578545 0.652231
BernoulliNB:
0.821964 0.572884 0.579677 0.658175
0.782055 0.519389 0.488820 0.596754
SGDClassifier:
0.794509 0.558166 0.580810 0.644495
0.752335 0.481461 0.534956 0.589584
LinearSVC:
0.800453 0.562129 0.591565 0.651382
RidgeClassifier:
0.781489 0.549958 0.574866 0.635437
0.806963 0.562695 0.598924 0.656194
over


#使用集合
LogisticRegression:
0.828191 0.580243 0.604585 0.671007
0.820549 0.590150 0.607982 0.672894
MultinomialNB:
0.816020 0.569771 0.582225 0.656005
BernoulliNB:
0.819700 0.567506 0.577413 0.654873
0.789414 0.529012 0.476083 0.598170
SGDClassifier:
0.815737 0.572035 0.595811 0.661194
0.754599 0.522219 0.550241 0.609020
LinearSVC:
0.828757 0.574016 0.595245 0.666006
RidgeClassifier:
0.806680 0.572035 0.585621 0.654779
0.819700 0.583923 0.597792 0.667138
over

#不使用集合
LogisticRegression:
0.804416 0.568072 0.588169 0.653552
0.817719 0.569488 0.596377 0.661194
MultinomialNB:
0.814888 0.562129 0.574299 0.650439
BernoulliNB:
0.827342 0.575998 0.584489 0.662610
0.784602 0.518823 0.498160 0.600528
SGDClassifier:
0.799887 0.560713 0.581376 0.647325
0.766770 0.505519 0.500991 0.591093
LinearSVC:
0.803849 0.559864 0.581376 0.648363
RidgeClassifier:
0.794509 0.547976 0.569771 0.637419
0.816303 0.559015 0.585621 0.653647
over

In [33]:
#VotingClassifier方案
import numpy as np
from sklearn.ensemble import VotingClassifier

from sklearn.linear_model.logistic import LogisticRegression
from sklearn.naive_bayes import MultinomialNB  
from sklearn.naive_bayes import BernoulliNB  
from sklearn.linear_model import SGDClassifier  

clf1 = LogisticRegression(C=2,penalty='l2')
clf2 = MultinomialNB(alpha=0.01)
clf3 = BernoulliNB(alpha=0.01)
clf4 = SGDClassifier(alpha=5e-05)

eclf = VotingClassifier(estimators=[('lr', clf1), ('mb', clf2), ('bb', clf3) , ('sg', clf4)], voting='hard')
eclf = eclf.fit(x_train, y_train_g)
predictions = eclf.predict( x_test )
vg = precision_score(y_test_g, predictions, average='micro')  
print vg

eclf = VotingClassifier(estimators=[('lr', clf1), ('mb', clf2), ('bb', clf3) , ('sg', clf4)], voting='hard')
eclf = eclf.fit(x_train, y_train_a)
predictions = eclf.predict( x_test )
va = precision_score(y_test_a, predictions, average='micro')  
print va

eclf = VotingClassifier(estimators=[('lr', clf1), ('mb', clf2), ('bb', clf3) , ('sg', clf4)], voting='hard')
eclf = eclf.fit(x_train, y_train_e)
predictions = eclf.predict( x_test )
vu = precision_score(y_test_e, predictions, average='micro')  
print vu
print( "%f %f %f %f" % ( vg , va , ve , (vg + va + ve) / 3.0 ) )


0.832080908526
0.583931295553
0.602519916379


NameError: name 've' is not defined

In [63]:
from sklearn.grid_search import GridSearchCV
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.linear_model import SGDClassifier  
from sklearn.linear_model import SGDClassifier, LogisticRegression,RidgeClassifier,PassiveAggressiveClassifier,Lasso,HuberRegressor

pipeline = Pipeline([
#    ('vect', TfidfVectorizer(max_features=10000,ngram_range=(1, 1), max_df=0.5, norm='l1', use_idf=True)),
#    ('clf', BernoulliNB(alpha=0.01)),
    
#    ('vect', TfidfVectorizer(max_df = 0.5 , ngram_range = (1, 2) , norm = 'l2')),
#    ('clf', MultinomialNB(alpha = 0.01))
    
#    ('vect', TfidfVectorizer(max_features=10000,ngram_range=(1, 1), max_df=0.5, norm='l1', use_idf=True)),
#    ('clf', SGDClassifier())

#    ('vect', TfidfVectorizer(max_features=10000,ngram_range=(1, 1), max_df=0.5, norm='l1', use_idf=True)),
#    ('clf', PassiveAggressiveClassifier())
    
    ('vect', TfidfVectorizer(max_df = 0.75)),
    ('clf', LogisticRegression(C=2,penalty='l2'))
])
parameters = {

    #'vect__max_df': ( 0.5 , 0.75 , 1 ),
    #'vect__stop_words': ('english', None ),
    'vect__max_features': ( 90000 , 100000 , 110000 ),
    #'vect__ngram_range': ((1, 1), (1, 2)),
    #'vect__use_idf': ( True , False ),
    #'vect__norm': ('l1', 'l2'),
    
    #'clf__penalty': ('l1', 'l2'),
    #'clf__C': ( 0.9 , 1 ),
    #'clf__loss': ( 'squared_hinge' , 'hinge' ),
    #'clf__n_iter': ( 20 , 30 )
    #'clf__alpha': ( 0.001 , 0.01 , 0.5 )  
}


grid = GridSearchCV( pipeline , parameters, n_jobs=2, verbose=1, scoring='accuracy', cv=4)
grid.fit(final_cut_data, Agelist)
#grid.fit(final_cut_data, Genderlist)
#grid.fit(final_cut_data, Edulist)

#Alllist
#grid.fit(x_train, y_train)
print(grid)
# summarize the results of the grid search
print(grid.best_score_)
best_parameters = grid.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print(' \t %s: %r' % (param_name, best_parameters[param_name]))

#predictions = grid_search.predict(X_test)


/usr/local/lib/python2.7/dist-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Fitting 4 folds for each of 3 candidates, totalling 12 fits


[Parallel(n_jobs=2)]: Done  12 out of  12 | elapsed: 15.5min finished


GridSearchCV(cv=4, error_score='raise',
       estimator=Pipeline(steps=[('vect', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=0.75, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=Tru...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=2,
       param_grid={'vect__max_features': (90000, 100000, 110000)},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=1)
0.60606608507
 	 vect__max_features: 100000


Agelist max_df: 0.75 max_features: 130000
Edulist max_df: 0.75 max_features: 170000
Genderlist max_df: 0.75 max_features: 90000

http://prozhuchen.com/2016/12/28/CCF%E5%A4%A7%E8%B5%9B%E6%90%9C%E7%8B%97%E7%94%A8%E6%88%B7%E7%94%BB%E5%83%8F%E6%80%BB%E7%BB%93/